
---

# 📘 Integration with Databases

FastAPI lets you easily connect and interact with databases using **SQLite**, **PostgreSQL**, or **ORMs** like **SQLAlchemy**.

This guide uses **SQLite (lightweight)** and **`sqlite3`** module for simplicity.

---

## ✅ Step 1: Install Requirements

```bash
pip install fastapi uvicorn
```

SQLite support is built-in with Python — no extra DB driver needed.

---

## ✅ Step 2: Create a Simple Database

```python
# db_setup.py
import sqlite3

def init_db():
    conn = sqlite3.connect("books.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS books (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            author TEXT NOT NULL
        )
    """)
    conn.commit()
    conn.close()

# Run this once
init_db()
```

---

## ✅ Step 3: Build FastAPI App

```python
# main.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import sqlite3

app = FastAPI()

class Book(BaseModel):
    title: str
    author: str

def get_db():
    conn = sqlite3.connect("books.db")
    conn.row_factory = sqlite3.Row
    return conn
```

---

## ✅ Step 4: Create API Endpoints

```python
@app.post("/books/")
def create_book(book: Book):
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute("INSERT INTO books (title, author) VALUES (?, ?)", (book.title, book.author))
    conn.commit()
    conn.close()
    return {"message": "Book added"}

@app.get("/books/")
def get_books():
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM books")
    books = cursor.fetchall()
    conn.close()
    return [dict(row) for row in books]
```

---

## ✅ Step 5: Run Server and Test

```bash
uvicorn main:app --reload
```

Test in browser:

* `POST /books/` → Add a book
* `GET /books/` → See list of books

---

## ✅ Bonus: Add GET by ID

```python
@app.get("/books/{book_id}")
def get_book(book_id: int):
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM books WHERE id = ?", (book_id,))
    book = cursor.fetchone()
    conn.close()
    if book:
        return dict(book)
    raise HTTPException(status_code=404, detail="Book not found")
```

---

## ✅ Summary Table

| Step           | Description                     |
| -------------- | ------------------------------- |
| `sqlite3`      | Simple, built-in DB for testing |
| `init_db()`    | One-time setup for schema       |
| `/books/` POST | Add a new book                  |
| `/books/` GET  | Get all books                   |
| `/books/{id}`  | Get a single book by ID         |

---

## 🧠 Tips

* Switch to **SQLAlchemy** or **Tortoise ORM** for larger projects.
* For production, use **PostgreSQL** or **MySQL**.
* Use connection pooling libraries (e.g., `databases`) for async performance.

---
